<h1>Modelo de Décio Bazin</h1>

A idéia é calcular o ranking (Bazin) das ações analisando os dados fundamentalistas de todas as empresas da bolsa B3.<br><br>
Para a análise, são utilizados princípios do Décio Bazin. Autor do livro: "Faça Fortuna Com Ações", que é tido como literatura indicada por Luis Barsi, o maior investidor na bolsa brasileira.<br>

<h3>Princípios utilizados:</h3>

1. Preço Justo (Bazin) > 1.5 * Preço. Preço Justo (Bazin) => Dividend Yield * 16.67 (Por: Décio Bazin)<br>
2. Dívida Bruta/Patrimônio < 0.5 (50%)<br>
3. Dividend Yield > 0.06 (6%)<br>
4. Média do Dividend Yield nos últimos 5 anos > 0.05 (5%)<br>
5. Mediana do Dividend Yield nos últimos 5 anos > 0.05 (5%)<br>
6. Pagamento positivo de dividendos nos últimos 5 anos<br>
7. Pagamento crescente de dividendos nos últimos 5 anos<br>
8. 0 < Payout < 1

## Bibliotecas

In [2]:
import sys, os
sys.path.extend([f'./{name}' for name in os.listdir(".") if os.path.isdir(name)])

import fundamentus
import bovespa
import backtest
import webbrowser

import pandas
import numpy
import urllib.parse

from math import sqrt
from decimal import Decimal

import http.cookiejar
import urllib.request
import json
import threading
import time
import pyperclip

import portfolios
import statistics

import warnings
warnings.filterwarnings("ignore")

## Funções

In [3]:
def populate_shares(year):
  globals()['year'] = year
  globals()['infos'] = {}
  
  if year == current_year():
    shares = bovespa.shares()
  else:
    shares = fundamentus.shares(year)
  
  shares = shares[shares['Cotação'] > 0]
  shares = shares[shares['Liquidez 2 meses'] > 0]
  shares['Ranking (Bazin)'] = 0
  
  fill_infos(shares)
  
  shares = add_ratings(shares)
  
  shares = reorder_columns(shares)
  
  return shares


# Captura a situação dos dividendos nos últimos 5 anos. (Captura do site: Suno Analitica)
# infos = {
#   'TRPL4': { 'constante': False, 'crescente': False },
#   'PETR4': { 'constante': False, 'crescente': False }
# }
def fill_infos(shares):
  cookie_jar = http.cookiejar.CookieJar()
  opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookie_jar))
  opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201'),
                       ('Accept', 'text/html, text/plain, text/css, text/sgml, */*;q=0.01')]
  tickers = list(shares.index)
  threads = [threading.Thread(target=fill_infos_by_ticker, args=(ticker,opener,)) for ticker in tickers]
  for thread in threads:
    thread.start()
  for thread in threads:
    thread.join()

def fill_infos_by_ticker(ticker, opener):
  infos[ticker] = {
    'media': 0.0,
    'mediana': 0.0,
    'constante': False,
    'crescente': False,
    'healthy_payout': False,
  }